In [ ]:
"""
Deep Learning on Graphs - ALTEGRAD - Nov 2023
MARENGO Matteo
matteo.marengo@ens-paris-saclay.fr
"""

import scipy.sparse as sp
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder

def normalize_adjacency(A):
    ############## Task 9

    ##################
    # your code here #
    # Add self-loops to the adjacency matrix
    A = A + sp.eye(A.shape[0])

    # Compute the diagonal degree matrix ~D
    D = sp.diags(A.sum(axis=1).flatten())

    # Compute the inverse square root of the diagonal degree matrix ~D^(-1/2)
    D_sqrt_inv = sp.diags(np.power(D.diagonal(), -0.5))

    # Compute the symmetric normalized adjacency matrix ^A
    A_normalized = D_sqrt_inv.dot(A).dot(D_sqrt_inv)

    ##################
    
	return A_normalized


def load_cora():
    idx_features_labels = np.genfromtxt("../data/cora.content", dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    features = features.todense()
    features /= features.sum(1).reshape(-1, 1)
    
    class_labels = idx_features_labels[:, -1]
    le = LabelEncoder()
    class_labels = le.fit_transform(class_labels)

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("../data/cora.cites", dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(class_labels.size, class_labels.size), dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    print('Dataset has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))

    return features, adj, class_labels


def sparse_to_torch_sparse(M):
    """Converts a sparse SciPy matrix to a sparse PyTorch tensor"""
    M = M.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((M.row, M.col)).astype(np.int64))
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
